In [12]:
import fitz
import pandas as pd
import re

In [13]:
pdf_document = fitz.open("li.pdf")
toc = pdf_document.get_toc(simple=True)

In [14]:

def extract_image_from_page(page, page_number):
    image_list = page.get_images(full=True)
    if image_list is None:
        return None
    else:
        images_name = []
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page_number+1}_{img_index + 1}.png"  # page number starts from 0 in fitz and 1 in pdf
            images_name.append(image_filename)
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)
        return images_name



In [15]:
max_title_length = max(len(toc[i][1]) for i in range(len(toc)))

for i in range(len(toc)):
    title = toc[i][1]  
    start_page = toc[i][2]
    if i != len(toc)-1:
        end_page = toc[i+1][2]
    else:
        end_page = len(pdf_document)

    print(f"{i:2}  {title:{max_title_length}}  {start_page:2}  {end_page:2}")


 0  1 Introduction                                          9  10
 1  1.1 Calcium Imaging                                    10  11
 2  1.1.1 Intracellular calcium dynamics and its modeling  11  13
 3  1.1.2 Calcium indicators                               13  15
 4  1.1.3 Data analysis and its difficulties               15  16
 5  1.2 Neural coding: from firing rate to correlation     16  17
 6  1.2.1 Correlation quantification                       17  18
 7  1.3 Question statement                                 18  19
 8  2 Materials and Methods                                19  19
 9  2.1 Software and hardware                              19  19
10  2.2 Fluorescence signal modeling                       19  20
11  2.2.1 Point process                                    20  22
12  2.2.2 Kernel                                           22  24
13  2.2.3 Exact integration as fast convolution            24  25
14  2.3 Signal characterization and correlation inference  25  25
15  2.3.1 

In [16]:
def separate_header(text):
    # Define a regex pattern to match the modified header format
    header_pattern = r'^\d+\s.*\s\d+$'  # Matches lines with a number, text, number
    
    # Find the header pattern in the text
    match = re.search(header_pattern, text, re.MULTILINE)
    
    if match:
        header = match.group(0)
        header_end_index = match.end()
        main_text = text[header_end_index:]
    else:
        # If no header pattern found, return the entire text as main text
        header = ''
        main_text = text
    
    return header, main_text

following version tries not to search through all lines.

def separate_header(text):
    # Define a regex pattern to match the modified header format
    header_pattern = r'^\d+\s.*\s\d+$'  # Matches lines with a number, text, number
    
    # Find the header pattern at the beginning of the text
    match = re.match(header_pattern, text)
    
    if match:
        header = match.group(0)
        header_end_index = match.end()
        main_text = text[header_end_index:].strip()  # Remove header and any leading/trailing whitespace
    else:
        # If no header pattern found, return the entire text as main text
        header = ''
        main_text = text.strip()  # Remove any leading/trailing whitespace
    
    return header, main_text

In [17]:
import pprint

pp = pprint.PrettyPrinter(width=200, compact=True)

In [48]:
parts_list = []

for i in range(len(toc)):
    print('iii', i)
    title = toc[i][1] 
    print('ttt-title', title) 
    start_page = toc[i][2]
    print('sss-start_page', start_page)
    if i != len(toc)-1:
        end_page = toc[i+1][2]
    else:
        end_page = len(pdf_document)   #!probelm of out of toc pages and generalization and end page
    
    print('eee-end_page', end_page)

    part_text = []
    part_images = []
    for j in range(start_page-1, end_page): #page number in pdf start from 1 and in fitz from 0
        print('jjj', j)
        page = pdf_document.load_page(j)
        page_images = extract_image_from_page(page, j)
        if page_images is not None:
            part_images.append(page_images)
        text = page.get_text("text")
        page_header, page_text = separate_header(text)
        #text = text.split("\n")[2:]   #regex is better
        #page_text = '\n'.join(text)
        part_text.append(page_text)   #incluse index of characers
        pp.pprint(part_text)
    
    if (i != len(toc)-1):   #!probelm of out of toc pages and generalization and end page
        print('True')
        part_text[0] = part_text[0].split(title)[1]   #to exclude text from previous part
        next_title = toc[i+1][1]
        part_text[-1] = part_text[-1].split(next_title)[0]  #to exclude text from next part
    else:
        print('False')
        part_text[0] = part_text[0].split(title)[1]     

    part_dict = {
        'title': title,
        'pages': f'{start_page}-{end_page}',
        'text': part_text,
        'images': part_images
    }

    parts_list.append(part_dict)

#pdf_document.close()

iii 0
ttt-title 1 Introduction
sss-start_page 9
eee-end_page 10
jjj 8
['\n'
 '1 Introduction\n'
 'Recording neural activity is critical for neuroscience research. Neural activity sculpts\n'
 'how neurons transfer information and interact with each other. Different markers or\n'
 'indicators can mediate it. Nowadays, a variety of techniques have been developed to\n'
 'measure brain activity, such as single-channel recording, multi-channel recording, lo-\n'
 'cal field potentials (LFPs), electroencephalogram (EEG), magnetoencephalography\n'
 '(MEG), functional magnetic resonance imaging (fMRI), positron emission tomog-\n'
 'raphy (PET), calcium imaging, optical imaging of intrinsic signals (OIS), volage-\n'
 'sensitive dye imaging (VSDI). These techniques can be classified into two main cat-\n'
 'egories: electrophysiology and imaging techniques. Electrophysiology (patch clamp,\n'
 'multi-channel recording, LFPs, EEG, MEG) measures and manipulates neuronal\n'
 'dynamics such as spike tra

In [47]:
len(toc)

43

In [45]:
l = 42
l != len(toc)-1 | l != len(toc)


False

In [31]:
a = pdf_document.load_page(8)
b = pdf_document.load_page(9)
atext = a.get_text("text").casefold()
btext = b.get_text("text").casefold()
text_test = []
text_test.append(atext)
text_test.append(btext)
atitle = toc[0][1].casefold()
btitle = toc[1][1].casefold()
text_test[0] = text_test[0].split(atitle)[1]
text_test[-1] = text_test[-1].split(btitle)[0]
text_test

['\n1\n',
 '1 introduction\n2\nfigure 1:\ntemporal resolution and spatial scale of imaging and electrophysi-\nology techniques.\nthe mesoscopic scale is represented by the oval shaded area\n[1].\nintra: intracellular recording, su/mu: single-unit/multi-unit recording,\nlfp: local field potential, mea: multi-electrode array, vsdi: voltage-sensitive dye\nimaging, ois: optical imaging of intrinsic signals, eeg/meg: electroencephalog-\nraphy/magnetoencephalography, fmri/pet: functional magnetic resonance imag-\ning/poistron emission tompgraphy.\n']

In [29]:
text_test[2]

IndexError: list index out of range

In [49]:
parts_list

[{'title': '1 Introduction',
  'pages': '9-10',
  'text': ['\nRecording neural activity is critical for neuroscience research. Neural activity sculpts\nhow neurons transfer information and interact with each other. Different markers or\nindicators can mediate it. Nowadays, a variety of techniques have been developed to\nmeasure brain activity, such as single-channel recording, multi-channel recording, lo-\ncal field potentials (LFPs), electroencephalogram (EEG), magnetoencephalography\n(MEG), functional magnetic resonance imaging (fMRI), positron emission tomog-\nraphy (PET), calcium imaging, optical imaging of intrinsic signals (OIS), volage-\nsensitive dye imaging (VSDI). These techniques can be classified into two main cat-\negories: electrophysiology and imaging techniques. Electrophysiology (patch clamp,\nmulti-channel recording, LFPs, EEG, MEG) measures and manipulates neuronal\ndynamics such as spike trains, EPSC, EPSP, and LFP. Imaging techniques (fMRI,\nPET, calcium imaging, O

In [50]:
pd.set_option('display.max_colwidth', 10000)
pd.DataFrame(parts_list)

,title,pages,text,images
0,1 Introduction,9-10,"[\nRecording neural activity is critical for neuroscience research. Neural activity sculpts\nhow neurons transfer information and interact with each other. Different markers or\nindicators can mediate it. Nowadays, a variety of techniques have been developed to\nmeasure brain activity, such as single-channel recording, multi-channel recording, lo-\ncal field potentials (LFPs), electroencephalogram (EEG), magnetoencephalography\n(MEG), functional magnetic resonance imaging (fMRI), positron emission tomog-\nraphy (PET), calcium imaging, optical imaging of intrinsic signals (OIS), volage-\nsensitive dye imaging (VSDI). These techniques can be classified into two main cat-\negories: electrophysiology and imaging techniques. Electrophysiology (patch clamp,\nmulti-channel recording, LFPs, EEG, MEG) measures and manipulates neuronal\ndynamics such as spike trains, EPSC, EPSP, and LFP. Imaging techniques (fMRI,\nPET, calcium imaging, OIS, VSDI) can capture neuronal activities and identify\nstructures and connections at different scales, from cortex areas and neurons to\ndendrites and spines. Each technology has advantages and disadvantages regarding\ninvasiveness and temporal and spatial resolution. Therefore, combining two or more\ntechnologies to reveal the complex spatiotemporal patterns of neural activity from\na single neuron to the population level is common. Figure 1 gives an overview of\ntemporal resolution and spatial scale.\n, \nFigure 1:\nTemporal resolution and spatial scale of imaging and electrophysi-\nology techniques.\nThe mesoscopic scale is represented by the oval shaded area\n[1].\nINTRA: intracellular recording, SU/MU: single-unit/multi-unit recording,\nLFP: local field potential, MEA: multi-electrode array, VSDI: voltage-sensitive dye\nimaging, OIS: optical imaging of intrinsic signals, EEG/MEG: electroencephalog-\nraphy/magnetoencephalography, fMRI/PET: functional magnetic resonance imag-\ning/poistron emission tompgraphy.\n]","[[], [image_10_1.png]]"
1,1.1 Calcium Imaging,10-11,"[\nAs one of the imaging techniques, calcium imaging has become more popular in the\nseveral decades. Neuronal spiking is assessed by measuring changes in fluorescence.\nCalcium imaging together with recombinant DNA technologies and microscopy is\na powerful tool in modulating and visualizing cellular and molecular interactions.\nIt can be used not only to observe neuronal activity but also to associate behav-\nioral characterization with physiological states. Under the field of view (FOV), it is\npossible to record the activities of hundreds of neurons. It also enables recording free-\nmoving animals in in-vivo experiments. Two-photon microscopy has excellent spatial\nresolution, but because the calcium signal is slow, two-photon calcium imaging has\nvery good spatial resolution but low temporal resolution. Although experiments’\nsetup and execution could be demanding, calcium imaging can be combined with\n, \nelectrophysiology to compensate the low temporal resolution. Figure 2 is an exam-\nple. Intra-cellular recording directly measures neuronal activities, usually taken as\nground truth. Calcium imaging records fluorescence signals as an indicator of neu-\nronal spiking activity, which is indirect. Fluorescence signals and neuronal spikes\ndiffer from each other. Figure 3 shows single action potential and its evoked calcium\nsignal, obtained from recording rats’ calyx of Held - a large synaptic terminal in\nbrainstem slice. We see action potential is quick (in ms) with a short decay, while\nfluorescence signal (in this case, Fura-2) is slow in several seconds with a longer\ndecay. The fluorescence signal is much noisier than the action potential. The cal-\ncium indicator Section 1.1.2 is one key point to facilitate functional imaging with\nalive animals. For example, with a cranial window over visual cortex of a transgenic\nmouse, it is able to image large scale of neuronal activities o